# Isensee

In [ ]:
!nvidia-smi -L

## Install requirements

In [ ]:
!pip install SimpleITK
!pip install -r https://raw.githubusercontent.com/Lasagne/Lasagne/master/requirements.txt
!pip install https://github.com/Lasagne/Lasagne/archive/master.zip
!pip install medpy
!pip install --upgrade batchgenerators

In [ ]:
import batchgenerators
batchgenerators_path = batchgenerators.__file__.split("/__init")[0]

In [ ]:
!wget https://raw.githubusercontent.com/MIC-DKFZ/batchgenerators/master/batchgenerators/dataloading/multi_threaded_augmenter.py -O $batchgenerators_path/dataloading/multi_threaded_augmenter.py

### pygpu

In [ ]:
!git clone https://github.com/Theano/libgpuarray.git
%cd libgpuarray
!mkdir Build

In [ ]:
%%bash
cd Build
# you can pass -DCMAKE_INSTALL_PREFIX=/path/to/somewhere to install to an alternate location
cmake .. -DCMAKE_BUILD_TYPE=Release # or Debug if you are investigating a crash
make
make install

In [ ]:
!python setup.py build
!python setup.py install

In [ ]:
%cd ..
!rm -rf libgpuarray

In [ ]:
!sudo ldconfig
#or restart the environment

## Import data

In [ ]:
!git clone https://github.com/manigalati/ACDC2017

In [ ]:
!wget https://raw.githubusercontent.com/matthuisman/files.matthuisman.nz/master/gdrivedl
!chmod +x gdrivedl
!./gdrivedl https://drive.google.com/open?id=1bxRj0zf-iMooYA4jUZS_zS4VD9NLYxE4 ../data/training.zip
!./gdrivedl https://drive.google.com/open?id=1L84oEmgc2Nd10bCBlaM7vkbl7nhrYC9I ../data/testing.zip

## Prepare dataset

In [ ]:
%cd ACDC2017

In [ ]:
!mkdir train_set & unzip -q ../../data/training.zip -d train_set/

In [ ]:
!unzip -q ../../data/testing.zip -d test_set/ 
!mv test_set/testing/testing test_set/tmp && rm -r test_set/testing && mv test_set/tmp test_set/testing

In [ ]:
!python dataset_utils.py -i train_set/training/ -out2d train_set_2d -out3d train_set_3d

## Training

In [ ]:
!mkdir out

### Training 2D

In [ ]:
!THEANO_FLAGS='floatX=float32,device=cuda0' python run_training_2D.py -f 0 -c UNet2D_config.py

### Training 3D

In [ ]:
!THEANO_FLAGS='floatX=float32,device=cuda0' python run_training_3D.py -f 0 -c UNet3D_config.py

## Testing

### Testing 2D

In [ ]:
!THEANO_FLAGS='floatX=float32,device=cuda0' python predict_test_2D_net.py -f 0 -c UNet2D_config.py

### Testing 3D

In [ ]:
!THEANO_FLAGS='floatX=float32,device=cuda0' python predict_test_3D_net.py -f 0 -c UNet3D_config.py

## Ensemble

In [ ]:
!python create_final_submission_files.py -c2d UNet2D_config.py -c3d UNet3D_config.py -o results